In [1]:
import re

In [2]:
class parser:
    def __init__(self, text):
        self.text = text
        self.sentences = self.__split_text()
        self.premises = []
        self.conclusion = ''
        self.patterns = {
            'implication': r"^if (.*), (then (.*)|(.*))",
            'junction': r"^(.*) (and|or) (.*)",
            'conclusion': r"therefore (.*)."
        }
        
    def __repr__(self):
        return '\033[1mText:\033[0m {} \n\n\033[1mPremises:\033[0m {} \n\n\033[1mConclusion:\033[0m {}'.format(self.text, self.premises, self.conclusion)
    
    def __str__(self):
        return '\033[1mText:\033[0m {} \n\n\033[1mPremises:\033[0m {} \n\n\033[1mConclusion:\033[0m {}'.format(self.text, self.premises, self.conclusion)
    
    def __split_text(self):
        return self.text.lower().split('. ')

    def __check_implication(self, sentence):
        matches = re.match(self.patterns['implication'], sentence)

        if not matches:
            return False

        p = matches.group(1)
        q = matches.group(3) if matches.group(3) else matches.group(2)

        self.premises.append({'p1': p, 'p2': q, 'type': 'implication', 'value': True})

        return True

    def __check_junction(self, sentence):
        matches = re.match(self.patterns['junction'], sentence)

        if not matches:
            return False

        p = matches.group(1)
        q = matches.group(3)
        
        if matches.group(2) == 'and':
            self.premises.append({'p1': p, 'p2': q, 'type': 'conjunction', 'value': True})
        else:
            self.premises.append({'p1': p, 'p2': q, 'type': 'disjunction', 'value': True})

        return True

    def __check_conclusion(self, sentence):
        matches = re.match(self.patterns['conclusion'], sentence)

        if not matches:
            return False

        p = matches.group(1)

        return p

    def parse(self):
        for sentence in self.sentences:
            temp = self.__check_implication(sentence)
            if not temp:
                temp = self.__check_junction(sentence)
                if not temp:
                    self.conclusion = self.__check_conclusion(sentence)
        result = [x['value'] for x in self.premises]
        if all(result):
            print(r"As long as this statment '{}' is true".format('\033[1m' + " and ".join(self.sentences[:-1]) + '\033[0m'), end='')
            print(r" Then '{}'.".format('\033[1m' + self.conclusion + '\033[0m'))
        return
    def simplification(self):
        for premise in self.premises:
            try:
                if premise['type'] == 'conjunction':
                    self.premises.append({'p1': premise['p1'], 'type': 'simplification', 'value': True})
                    self.premises.append({'p1': premise['p2'], 'type': 'simplification', 'value': True})
            except:
                pass
    def modus_ponens(self):
        for premise in self.premises:
            try:
                if premise['type'] == 'implication':
                    for prem in self.premises:
                        if prem['type'] == 'simplification':
                            if prem['p1'] == premise['p1']:
                                self.premises.append({'p1': premise['p2'], 'type': 'modus ponens', 'value': True})
            except:
                pass
    def describe(self):
        print('\033[1mText:\033[0m {} \n'.format(self.text))
        print('*'*50)
        print('\033[1mPremises:\033[0m')
        for i, premise in enumerate(self.premises):
            print('{}- {}'.format(i+1, premise))
            if premise['type'] == 'implication':
                print('\033[1mp1 -> p2\033[0m')
            elif premise['type'] == 'conjunction':
                print('\033[1mp1 ^ p2\033[0m')
            elif premise['type'] == 'disjuntion':
                print('\033[1mp1 ∨ p2\033[0m')
            else:
                print('\033[1mp1\033[0m')
            print('*'*50)
        print('\n\033[1mConclusion:\033[0m {}'.format(self.conclusion))
        if len(self.premises) == 7:
            print('Using premises\n \033[1m6- {} and\n 7- {} \033[0m\n using \033[1mdisjunctive syllogism\033[0m.\n Thus, {}'.format(self.premises[6]['p1'], self.premises[5]['p1'], self.conclusion))


In [3]:
text = "If today is Tuesday, I have a test in English or Science. If my English Professor is absent, then \
I will not have a test in English. Today is Tuesday and my English Professor is absent. \
Therefore I have a test in Science."

parser = parser(text)

parser.parse()

As long as this statment 'if today is tuesday, i have a test in english or science and if my english professor is absent, then i will not have a test in english and today is tuesday and my english professor is absent' is true Then 'i have a test in science'.


In [4]:
parser.describe()

Text: If today is Tuesday, I have a test in English or Science. If my English Professor is absent, then I will not have a test in English. Today is Tuesday and my English Professor is absent. Therefore I have a test in Science. 

**************************************************
Premises:
1- {'p1': 'today is tuesday', 'p2': 'i have a test in english or science', 'type': 'implication', 'value': True}
p1 -> p2
**************************************************
2- {'p1': 'my english professor is absent', 'p2': 'i will not have a test in english', 'type': 'implication', 'value': True}
p1 -> p2
**************************************************
3- {'p1': 'today is tuesday', 'p2': 'my english professor is absent', 'type': 'conjunction', 'value': True}
p1 ^ p2
**************************************************

Conclusion: i have a test in science


In [5]:
parser.simplification()

In [6]:
parser.describe()

Text: If today is Tuesday, I have a test in English or Science. If my English Professor is absent, then I will not have a test in English. Today is Tuesday and my English Professor is absent. Therefore I have a test in Science. 

**************************************************
Premises:
1- {'p1': 'today is tuesday', 'p2': 'i have a test in english or science', 'type': 'implication', 'value': True}
p1 -> p2
**************************************************
2- {'p1': 'my english professor is absent', 'p2': 'i will not have a test in english', 'type': 'implication', 'value': True}
p1 -> p2
**************************************************
3- {'p1': 'today is tuesday', 'p2': 'my english professor is absent', 'type': 'conjunction', 'value': True}
p1 ^ p2
**************************************************
4- {'p1': 'today is tuesday', 'type': 'simplification', 'value': True}
p1
**************************************************
5- {'p1': 'my english professor is absent', 'type': 'simpli

In [7]:
parser.modus_ponens()

In [8]:
parser.describe()

Text: If today is Tuesday, I have a test in English or Science. If my English Professor is absent, then I will not have a test in English. Today is Tuesday and my English Professor is absent. Therefore I have a test in Science. 

**************************************************
Premises:
1- {'p1': 'today is tuesday', 'p2': 'i have a test in english or science', 'type': 'implication', 'value': True}
p1 -> p2
**************************************************
2- {'p1': 'my english professor is absent', 'p2': 'i will not have a test in english', 'type': 'implication', 'value': True}
p1 -> p2
**************************************************
3- {'p1': 'today is tuesday', 'p2': 'my english professor is absent', 'type': 'conjunction', 'value': True}
p1 ^ p2
**************************************************
4- {'p1': 'today is tuesday', 'type': 'simplification', 'value': True}
p1
**************************************************
5- {'p1': 'my english professor is absent', 'type': 'simpli